#首先从elasticsearch中选取提取符合条件的数据

In [2]:
import numpy as np
import elasticsearch
from elasticsearch_dsl import DocType
from elasticsearch_dsl.connections import connections
from elasticsearch_dsl.query import *
 
es = elasticsearch.Elasticsearch("192.168.*.*:8888", time_out=30)
connections.add_connection('default', es)
 
    
start = '2018-07-25T00:00:00'
end = '2018-08-03T23:59:59'
query1 = {
        '_source':
            {
                'include':['*','*','*'] 
            },
        'query':
            {
                'bool':{
                    'must':[{
                        'range':{
                            'begin':{
                                "gte": start,
                                "lte": end
                            }    
                        }
                    },
                    {
                        'exists':{
                            'field':'label'
                        }
                    }]
                }
            }                
        }
res = es.search(index='*', doc_type='*',size = 3000, body=query1)['hits']['hits']

ModuleNotFoundError: No module named 'elasticsearch'

将是字符串类型的数据映射成数字   
Q：简单的映射为0，1，2，3可行吗？   
Q：既然能映射成0，1，2，3，为什么不能映射成0，100，200，300？这两种映射方案会影响后来的决策结果吗？   
eg：假设说有一个字段rca，它的取值为‘connection normal’，‘Auth cost ***’，‘dhcp cost ***’，other,...,将其映射为0，1，2，3或者是映射为0，100，200，300，最后的决策结果是一样的吗？

In [ ]:
import pandas as pd
data = []
for i in range(len(res)):
    data.append(res[i]['_source'])
df = pd.DataFrame(data)
df['rca_num']= df.rca.apply(lambda x: 0 if x=='connection normal' 
                            else 1 if re.match( r"Auth cost", x,flags=0) is not None
                            else 2 if re.match( r"dhcp cost", x,flags=0) is not None 
                            else 3)

用one-hot编码，处理特征值为字符串的特征

In [4]:
one_hot_feature = ['*','*']
X = df[one_hot_feature] # or X = df.drop['*','*']
y = df['label_num']
# X_train=X_train.values.astype(float)
# y_train=y_train.values.astype(float)
# X_test = X_test.values.astype(float)
# y_test=y_test.values.astype(float)
features = X.columns.tolist() #得到特征值
mylabel = df['label_num']
targets = list(set(mylabel.tolist())) # 得到类别名称

NameError: name 'df' is not defined

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(X) 
X_encode = enc.transform(X).toarray()

demean,PCA

决策树训练

GridSearchCV得到最佳参数

查看训练结果

In [ ]:
report = classification_report(y_test,y_predict,target_names=target_names)# precision, recall, f1_score
print report

混淆矩阵，学习曲线的绘制

对特征的重要性进行排序，选择比较重要的特征再进行训练，看看训练结果如何

In [ ]:
# GBDT
feature_index=sorted(enumerate(GraBoostClf.feature_importances_), key=lambda x:x[1],reverse=True)

绘制带有网格线的重要性曲线

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
fig = plt.figure()
ax1 = fig.add_subplot(1,1,1)
spacing = 1 # This can be your user specified spacing. 
minorLocator = MultipleLocator(spacing)
ax1.plot(np.arange(1,len(feature_index)+1),
         [np.sum(sort_gbdt_fea [:i]) for i in range(1,len(feature_index)+1)])
# Set minor tick locations.
ax1.yaxis.set_minor_locator(minorLocator)
ax1.xaxis.set_minor_locator(minorLocator)
# Set grid to use minor tick locations. 
ax1.grid(which = 'minor',linestyle='--')
plt.show()

选取重要特征，组成新的数据集

In [ ]:
X_train2 = X_train.loc[:,new_columns]

In [ ]:
# RF，用oob_score, 但是我们不知道测试数据集到底是哪些，编写函数实现，但貌似不对
from sklearn.utils import check_random_state,check_array
# X1 = check_array(X1, accept_sparse="csc", dtype=DTYPE)
# y = check_array(y, accept_sparse='csc', ensure_2d=False, dtype=None)
def generate_sample_indices(random_state, n_samples):
    """Private function used to _parallel_build_trees function."""
    random_instance = check_random_state(random_state)
    sample_indices = random_instance.randint(0, n_samples, n_samples)
 
    return sample_indices
 
 
def generate_unsampled_indices(random_state, n_samples):
    """Private function used to forest._set_oob_score function."""
    sample_indices = generate_sample_indices(random_state, n_samples)
    sample_counts = np.bincount(sample_indices, minlength=n_samples)
    unsampled_mask = sample_counts == 0
    indices_range = np.arange(n_samples)
    unsampled_indices = indices_range[unsampled_mask]
 
    return unsampled_indices
unsampled_indices = generate_unsampled_indices(666, 2000)#666=RF中参数random_state
len(unsampled_indices)

In [ ]:
# 绘制曲线，report 中为str 类型，不知道怎么取出来,所以就自己编写tpr，fpr，recall，precision函数
def tpr(y_true,y_pred, label):
    '''
    pos_label 此时必须是从1开始的int类型
    '''
    my_cmp = confusion_matrix(y_true, y_pred)
    index = [_ for _ in range(len(my_cmp))]
    my_tprs = []
    for pos_label in label:
        index_tmp = index[:]
        index_tmp.pop(pos_label-1)     
        fn = np.sum(my_cmp[pos_label-1][index_tmp])
        tp = my_cmp[pos_label-1][pos_label-1]
        try:
            my_tpr = float(tp) / (fn+tp)
        except:
            my_tpr = 0
        my_tprs.append(my_tpr)
    return my_tprs
    
        
def fpr(y_true,y_pred, label):
    myconfusion_matrix = confusion_matrix(y_true, y_pred)
    index = [_ for _ in range(len(myconfusion_matrix))]
    my_fprs = []
    for pos_label in label:
        index_tmp = index[:]
        index_tmp.pop(pos_label-1)
        cmp_tmp = myconfusion_matrix[index_tmp]
        cmp_tmp1 = cmp_tmp[:,index_tmp]
        tn = np.sum(cmp_tmp1)
        cmp_tmp2 = myconfusion_matrix[:,pos_label-1]
        cmp_tmp3 = cmp_tmp2[index_tmp]
        fp = np.sum(cmp_tmp3)
        try:
            my_fpr = float(fp) / (tn+fp)
        except:
            my_fpr = 0
        my_fprs.append(my_fpr)
    return my_fprs
def myrecall(y_true, y_pred, label):
    my_cmp = confusion_matrix(y_true, y_pred)
    index = [_ for _ in range(len(my_cmp))]
    my_recalls = []
    for pos_label in label:
        index_tmp = index[:]
        index_tmp.pop(pos_label - 1)
        fn = np.sum(my_cmp[pos_label - 1][index_tmp])
        tp = my_cmp[pos_label - 1][pos_label - 1]
        try:
            my_recall = float(tp) / (fn + tp)
        except:
            my_recall = 0
        my_recalls.append(my_recall)
    return my_recalls
 
def myprecision(y_true, y_pred, label):
    my_cmp = confusion_matrix(y_true, y_pred)
    index = [_ for _ in range(len(cmp))]
    my_precisions = []
    for pos_label in label:
        tp = my_cmp[pos_label - 1][pos_label - 1]
        index_tmp = index[:]
        index_tmp.pop(pos_label - 1)
        cmp_tmp1 = my_cmp[:,pos_label - 1]
        cmp_tmp2 = cmp_tmp1[index_tmp]
        fp = np.sum(cmp_tmp2)
        try:
            my_precision = float(tp) / (tp + fp)
        except:
            my_precision = 0
        my_precisions.append(my_precision)
    return my_precisions
 
# f1_score = 2 / [(1/precision) + (1/recall)],
def myf1_score(y_true, y_pred, label):
    my_precisions = myprecision(y_true, y_pred, label)
    my_recalls = myrecall(y_true, y_pred, label)
    precision_reciprocal = [1/x for x in my_precisions]
    recalls_reciprocal = [1/x for x in my_recalls]
    temp = [precision_reciprocal[_] + recalls_reciprocal[_] for _ in range(len(precision_reciprocal))]
    myf1_scores = [2/_ for _ in temp]
    return myf1_scores

决策树的绘制

In [ ]:
# 绘决策树
from sklearn import tree
from IPython.display import display, Image
dot_data=tree.export_graphviz(dt_clf2,out_file=None,
                    feature_names=select_features,
                    class_names=class_names,
                    filled=True,rounded=True,
                    special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data)
display(Image(graph.create_png()))